In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_extract, to_date
import pyspark.sql.functions as F
import matplotlib.pyplot as plt
import seaborn as sns

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("example_app") \
    .getOrCreate()

StatementMeta(, 55d20bc7-f91b-4d16-ac82-b43ce88b1a19, 10, Finished, Available, Finished)

## Data Quality Functions

###### Function 1 - Primary key validation

In [9]:

from pyspark.sql.window import Window

def keyValidation(df, subset_columns):
    # Count the number of duplicate rows based on the subset of columns
    duplicate_count = df.groupBy(subset_columns).count().filter(F.col("count") > 1).count()
    return duplicate_count

def handle_duplicates(df, subset_columns, strategy='remove', keep='last'):
    if strategy == 'remove':
        if keep == 'first':
            return df.dropDuplicates([subset_columns])
        elif keep == 'last':
            window_spec = Window.partitionBy(subset_columns).orderBy(F.monotonically_increasing_id())
            return df.withColumn("row_num", F.row_number().over(window_spec)) \
                     .filter(F.col("row_num") == 1) \
                     .drop("row_num")
        else:
            raise ValueError("Invalid 'keep' parameter. Use 'first' or 'last'.")
    
    elif strategy == 'flag':
        window_spec = Window.partitionBy(subset_columns)
        return df.withColumn("is_duplicate", F.count("*").over(window_spec) > 1)
    
    elif strategy == 'aggregate':
        # You need to specify the aggregation functions for each column
        # For example, let's assume you want to take the first value for each column
        agg_exprs = [F.first(col).alias(col) for col in df.columns if col not in subset_columns]
        return df.groupBy(subset_columns).agg(*agg_exprs)
    
    else:
        raise ValueError("Invalid strategy provided. Use 'remove', 'flag', or 'aggregate'.")

StatementMeta(, 55d20bc7-f91b-4d16-ac82-b43ce88b1a19, 11, Finished, Available, Finished)

###### Function 2 - Null value validation

## Customer

In [10]:
# Read the Parquet file
df_customer = spark.read.parquet("Files/SalesLT.Customer_W01_data_delta_bronze")

print((df_customer.count(), len(df_customer.columns)))
print('----------')
df_customer.printSchema()

StatementMeta(, 55d20bc7-f91b-4d16-ac82-b43ce88b1a19, 12, Finished, Available, Finished)

(100, 15)
----------
root
 |-- CustomerID: long (nullable = true)
 |-- NameStyle: long (nullable = true)
 |-- Title: string (nullable = true)
 |-- FirstName: string (nullable = true)
 |-- MiddleName: string (nullable = true)
 |-- LastName: string (nullable = true)
 |-- Suffix: string (nullable = true)
 |-- CompanyName: string (nullable = true)
 |-- SalesPerson: string (nullable = true)
 |-- EmailAddress: string (nullable = true)
 |-- Phone: string (nullable = true)
 |-- PasswordHash: string (nullable = true)
 |-- PasswordSalt: string (nullable = true)
 |-- rowguid: string (nullable = true)
 |-- ModifiedDate: string (nullable = true)



In [14]:

duplicate_count = keyValidation(df_customer,'CustomerID')

if duplicate_count > 0:
    print(f"The primary key is not unique.")
    df_customer = handle_duplicates(df_customer,'CustomerID')
    
else:
    print(f"No duplicate values found, the primary key is unique.")


StatementMeta(, 55d20bc7-f91b-4d16-ac82-b43ce88b1a19, 16, Finished, Available, Finished)

No duplicate values found, the primary key is unique.
